In [ ]:
#Install the necessary packages

!pip install yfinance
!pip install matplotlib==3.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
^C


In [ ]:
#Standard packages
import numpy as np
import pandas as pd

#Dates
from datetime import datetime, timedelta

#Finance packages
import yfinance as yf

#Statistics
from scipy.stats import binom

#Plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from matplotlib import rcParams

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

A financial option is a contract between a buyer and a seller that gives the buyer the right, but not the obligation, to buy or sell an underlying asset, such as a stock, at a specific price (**strike price**, $K$) on or before a specific date (**expiration date**, $T$). The seller, or option writer, is obligated to fulfill the terms of the contract if the buyer chooses to exercise their option. Options can be used for a variety of purposes, such as hedging risk or speculating on the price movement of an asset. There are two main types of options: **call** options, which give the buyer the right to buy the underlying asset, and **put** options, which give the buyer the right to sell the underlying asset. The options are buyed at a given price, $C_0$.

There are different types of options with different obligations, but we will focus on the European ones, which can only be executed at the expiration date.

Mathematically, we can evaluate the payoff of an (european) option at the expiration date $T$ as function of the value of the underlying asset at that moment, $Y(T)$, the strike price $K$ and the initial price, $C_0$.

The payoff of a call option is the difference between the price of the underlying asset and the strike price of the option

$$P_c(Y,T) = max(Y(T) - K - C_0, -C_0)$$

The payoff of a put option is the difference between the strike price of the option and the price of the underlying asset

$$P_p(Y,T) = max(K - Y(T)-C_0, -C_0)$$

It's important to note that these payoffs are only realized if the option is exercised and the underlying asset is bought or sold at the strike price. Also, the above equations are for European Options, so for other options the payoffs can be different.

#The binomial model

The binomial option pricing model is a mathematical model that is used to determine the *fair value* of an option. It is based on the idea that the underlying asset can move to one of two possible states (up or down) at the end of each time period. The model uses a recursive algorithm to calculate the value of the option at each point in time, taking into account the probability of the underlying asset moving up or down and the resulting payoffs from the option. In this model, the fair price of the option after a time $t=N\Delta t$ (so $N$ steps) is given by

$$C_0=e^{-rt}\sum_{k=0}^N\left(\begin{array}{c}N \\
k\end{array}\right)p_e^k\left(1-p_e\right)^{N-k}max(u^kd^{N-k}Y_0-K, 0)$$

with $p_e=\frac{e^{r\Delta t}-d}{u-d}$ and $u, d$ being the factor in which the price of the option can change *up* or *down*, respectively.

Indeed, a closed equation for the *fair price* of the option after $N$ steps can be found in terms of the cumulative distribution function of the binomial distribution, $F(x; N, p)$,

$$C_0=Y_0F(N-a, N, q_e')- e^{-rt}KF(N-a, N, q_e)$$

where

* $a=\left\lceil\frac{\ln(K/Y_0d^N)}{\ln(u/d)}\right\rceil$ where $\lceil x\rceil$ is the ceil function, which returns the smallest integer $i$ such that $i\geq x$

* $q_e = 1-p_e$

* $q_e'=1-ue^{-r\Delta t}p_e$

* $F(N-a,N,q_e)$ is the cumulative distribution of the binomial

In practice, $u$ and $d$ are determined from the volatility of the underlying asset.

# Exercise 1. Option Pricing using the binomial model

1. Implement the binomial model for option pricing using the first of the formulas above

  * I give you the implementation of the second formula, the "analytical" one.

2. Compare your implementation to the analytical one by giving some values to the parameters and runing the functions.

3. Use the binomial model to price real options of BP.

  * Compare your predicted fair values of the options with the traded ones. Is there a correspondance?

4. Repeat the previous step now with options from Tesla

  * What do you observe? Why do you think this is happening?



**1. Complete the functions below to implement the binomial model for a call option.**

*Clue: use the `numpy.ceil()` method to obtain $a$*

*Clue: use the `scipy.stats.binom` function to compute the CDF of the binomiald distribution. The function is alrealy loaded as `binom`*

In [ ]:
def comb(n, i): #Returns the result of n combined with i

    return np.math.factorial(n) / (np.math.factorial(n-i)*np.math.factorial(i))

def call_price_binomial_computational(N, Y0, K, r, dt, u, d):

  #CODE

def call_price_binomial_analytical(N, Y0, K, r, dt, u, d):

  #CODE

IndentationError: ignored

**2. Compare computational and theoretical methods**

In [ ]:
#Initial value of the underlying asset
Y0  = 100

#Volatility of the underlying asset
sigma = 0.5

#Strike price
K = 110

#Interest rate (zero for simplicity)
r = 0.0

#Time step
dt = 0.1

#Up and down price change factors
u = np.exp(sigma*np.sqrt(dt))
d = np.exp(-sigma*np.sqrt(dt))

#Steps
steps = 100

#Compute the fair prices for each step (N from 0 to steps) using the computational and analytical methods

#CODE

#Plot the prices according to both methods

#CODE

**3. Price Real Options from BP**

Run the following cell to download the data.

**It will take a while to download, be patient.**

Then, price each **Call** option using the binomial model and compare the results with the real traded prices.

**Note that we download data for Call and Put options, but we only want to price the Call ones**

In [ ]:
def options_data(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        # opt = pd.DataFrame().append(opt.calls).append(opt.puts) ## deprecated
        opt = pd.concat((opt.calls, opt.puts))
        opt['expirationDate'] = e
        # options = options.append(opt, ignore_index=True) ## deprecated
        options = pd.concat((options,opt), ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.today()).dt.days / 365

    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)

    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)

    # Drop unnecessary and meaningless columns
    #options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'bid', 'ask', 'inT'])
    options = options[["lastTradeDate", "strike", "lastPrice", "expirationDate", "impliedVolatility", "openInterest", "CALL"]]

    #Add underlying price of asset for each tradeDate
    print("Matching last traded dates with underlying prices... this might take a while...")

    underlying_prices = []

    tot = len(options)

    i = 0

    for date in options["lastTradeDate"]:

      pct = 100*i/tot

      if np.round((pct % 2), 2) == 0.0:

        print("Completed: %.2f %%" % pct)

      Y0 = yf.download(asset, start=date, end=date, progress=False, period="1d")["Adj Close"].values

      if len(Y0) == 0:

        underlying_prices.append(np.nan)

      else:

        underlying_prices.append(Y0[0])

      i += 1

    options["underlying_price"] = underlying_prices

    return options

asset = "BP"

df = options_data(asset)

df = df.dropna()

df["lastTradeDate"] = pd.to_datetime(df["lastTradeDate"])
df["expirationDate"] = pd.to_datetime(df["expirationDate"])

df

In [ ]:
#CODE

**4. Repeat the last step for options from Tesla**

*Note: Downloading these options is quite slow, as we have to match the original value of the asset for each traded date in our dataset and Tesla has many options being traded. Anyway, I did this for you so that you just need to load a CSV file*

In [ ]:
df = pd.read_csv("drive/MyDrive/Colab Notebooks/Econophysics/Practical Cases/Options_TSLA.csv")

df = df.drop("Unnamed: 0", axis=1)
df = df.dropna()

df["lastTradeDate"] = pd.to_datetime(df["lastTradeDate"])
df["expirationDate"] = pd.to_datetime(df["expirationDate"])

df

In [ ]:
#CODE

# Exercise 2. Implement the Black-Scholes model

The Black-Scholes model is a mathematical model used to determine the fair value of a European call option. The model was first published in 1973 by Fischer Black and Myron Scholes. The model makes several assumptions about the market, such as that the underlying asset follows a lognormal distribution, the options are European, and markets are frictionless.

The value of a call option is calculated using the following formula:

$$ C(t, T) = Y(t)\mathcal{N}(d_1) - Ke^{-r(T-t)}\mathcal{N}(d_2) $$

Where:

* $C$ is the value of the call option
* $Y(t)$ is the current price of the underlying asset
* $K$ is the strike price of the option
* $r$ is the risk-free interest rate
* $T$ is the time to expiration in years
* $\sigma$ is the volatility of the underlying asset
* $\mathcal{N}(x)$ is the cumulative probability density function of the standard normal distribution
* $d_1, d_2$ are defined as:

$$ d_1 = \frac{ln(\frac{Y(t)}{K}) + (r + \frac{\sigma^2}{2})(T-t)}{\sigma\sqrt{T-t}} $$

$$ d_2 = d_1 - \sigma\sqrt{T-t} $$

It's important to note that the Black-Scholes model makes certain assumptions about the market, such as the underlying asset follows a lognormal distribution and the options are European and markets are frictionless, so the results may not be accurate for all situations.

**1. Implement the Black-Scholes model**

In [ ]:
from scipy.stats import norm

def black_scholes(t, S, K, T, r, sigma):

    #CODE

    return call

**2. Compute the price of an option with the given parameters using the Black-Scholes model and the binomial model (with both analytical and computational methods)**

 * I already give you this done

 * Plot the results for the values at different times. What do you observe?

In [ ]:
S = 100 #price of the underlying asset
K = 110 #strike price of the option
T = 100 #time to expiration
r = 0.0 #risk-free interest rate
sigma = 0.5 #volatility of the underlying asset
t = 0

#Time step
dt = 0.25

#Up and down price change factors
u = np.exp(sigma*np.sqrt(dt))
d = np.exp(-sigma*np.sqrt(dt))

#Steps
steps = int(T/dt)

#Compute the fair prices for each step (N from 0 to steps) using the computational and analytical methods
C0_binom_comp = [call_price_binomial_computational(N, Y0, K, r, dt, u, d) for N in range(0, steps)]

C0_binom_anal = [call_price_binomial_analytical(N, Y0, K, r, dt, u, d) for N in range(0, steps)]

C0_BS = [black_scholes(t, S, K, T, r, sigma) for t in np.linspace(T, 0, 100)]

time_binom = np.linspace(0, T, steps)

plt.plot(C0_BS, lw=3, label="Black-Scholes")
plt.plot(time_binom, C0_binom_comp, lw=3, label="Binomial computational")
plt.plot(time_binom, C0_binom_anal, lw=3, label="Binomial analytical")

plt.legend()

**3. Repeat the previous step decreasing the time-step. Plot it for 4 different time steps, each one smaller than the previous one.**

  * What do you observe?

In [ ]:
S = 100 #price of the underlying asset
K = 110 #strike price of the option
T = 100 #time to expiration
r = 0.0 #risk-free interest rate
sigma = 0.5 #volatility of the underlying asset
t = 0

#CODE